In [ ]:
import h5py
import os
import re

import numpy as np

In [ ]:
HOME_PATH = '/home/shubhams/Hercules/kidstube-data/'
READ_DATASET = 'processed/new_annotated_data.hdf5'

SECONDS_PER_CLIP = 3
NUM_CLASSES = 4

WRITE_DATASET = 'processed/aggregate_{0}_sec/unbalanced_data.hdf5'.format(SECONDS_PER_CLIP)

In [ ]:
def read_hdf5(name, path=HOME_PATH + READ_DATASET):
    f = h5py.File(path, 'r')
    return f[name][()]

In [ ]:
frames = read_hdf5('frames')
annotations = read_hdf5('annotations')

In [ ]:
reshaped_frames = np.reshape(frames, (int(frames.shape[0] / SECONDS_PER_CLIP), frames.shape[1] * SECONDS_PER_CLIP, frames.shape[2]))
print(reshaped_frames.shape)

In [ ]:
reshaped_annotations = []
for i in range(0, annotations.shape[0], SECONDS_PER_CLIP):
    reshaped_annotations.append(np.bincount(annotations[i:i+SECONDS_PER_CLIP].astype(int)).argmax())

In [ ]:
reshaped_annotations = np.array(reshaped_annotations)
print(reshaped_annotations.shape)

In [ ]:
label_check_ctr = [0 for i in range(NUM_CLASSES)]
for i in range(reshaped_annotations.shape[0]):
    label_check_ctr[int(reshaped_annotations[i])] += 1
print(label_check_ctr)

In [ ]:
def save_data(frames, annotations, path=HOME_PATH+WRITE_DATASET):
    if not os.path.exists(os.path.dirname(HOME_PATH+WRITE_DATASET)):
        os.makedirs(os.path.dirname(HOME_PATH+WRITE_DATASET))
    with h5py.File(path, 'w') as f:
        f.create_dataset('frames', data=frames, compression='gzip')
        f.create_dataset('annotations', data=annotations, compression='gzip')

In [ ]:
save_data(reshaped_frames, reshaped_annotations)